<a href="https://colab.research.google.com/github/seoes/exhibition-visitor-tracker/blob/main/3-perspective-transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 원근 변환 위한 점 정의

In [5]:
import numpy as np
import cv2

width = 400
height = 2000

src_points = np.float32([[378,260],[688,260],[1080,1745],[0,1745]])
dst_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

M = cv2.getPerspectiveTransform(src_points, dst_points)

In [16]:
from collections import defaultdict

filtered_each_coord = list()
temp_array = list()

with open('/content/drive/MyDrive/results/3-filtered_coord.txt', 'r') as f:
    for line in f:
        if line.strip():  # 빈 줄이 아니면
            tuple_elements = line.strip().split(', ')
            temp_list = list(map(lambda x: int(x) if x.isdigit() else x, tuple_elements))
            temp_list[2] = float(temp_list[2])
            temp_tuple = tuple(temp_list)
            temp_array.append(temp_tuple)
        else:  # 빈 줄이면 현재까지의 temp_array를 loaded_nested_data에 추가하고 temp_array 초기화
            filtered_each_coord.append(temp_array)
            temp_array = list()
    if temp_array:  # 파일의 마지막에 빈 줄이 없는 경우 처리
        filtered_each_coord.append(temp_array)

# 객체별 경로를 저장할 defaultdict 생성
trajectories = defaultdict(list)

for frame_coords in filtered_each_coord:
  for coord in frame_coords:
    frame,obj_id, conf, x,y = coord
    # 좌표에 원근 변환 적용
    transformed_coord = cv2.perspectiveTransform(np.array([[x, y]], dtype='float32').reshape(-1, 1, 2), M)
    # 변환된 좌표를 객체 ID에 맞게 리스트에 추가
    trajectories[obj_id].append((frame, transformed_coord[0][0][0], transformed_coord[0][0][1]))

with open("/content/drive/MyDrive/results/4-trajectories.txt", "w") as file:
    for obj_id, coords in trajectories.items():
        for coord in coords:
            frame, x, y = coord
            file.write(f"{obj_id}, {frame}, {x}, {y}\n")

### 비디오 다운로드

In [20]:
import os
HOME = os.getcwd()
%cd {HOME}

!pip install gdown
# !gdown --id 1-Jt6B5D68UqHc5dcP8EwtRoIdjKENlz_ -O ./models/pretrained_model.pth
# !gdown --id 1vcKRPpv7ve1St-0rynhGa-H3lHMyCNt9 -O exhibition-64-sample.mp4
!gdown --id 1jEhErOcFKt7JrliQFzMar5yerHkTeu09 -O exhibition-64-full.mp4

FULL_VIDEO_64 = f"{HOME}/exhibition-64-full.mp4"

/content
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1jEhErOcFKt7JrliQFzMar5yerHkTeu09

but Gdown can't. Please check connections and permissions.


In [ ]:
# 비디오를 로드
cap = cv2.VideoCapture(FULL_VIDEO_64)

# 첫 프레임을 읽기
ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
warped_image = cv2.warpPerspective(frame, M, (width, height))